In [1]:
import pandas as pd
import csv
import sys, json
from sqlalchemy import create_engine
import psycopg2
import pymongo
import datetime
import numpy as np
np.random.seed(1)
import requests
from pprint import pprint
import matplotlib.pyplot as plt
from flask import jsonify

import company_domain
company_domain=company_domain.company_domain

# Extraction

##  Go to the files path to extract Inc 5000 Americas Fastest Growing Companies list

In [2]:
# The path to our CSV files to extract all Inc 5000 company list

# 2019 Inc 5000 company list
Inc_2019_csv = "Resources/inc5000-2019.csv"
# 2018 Inc 5000 company list
Inc_2018_csv = "Resources/inc5000-2018.csv"
# 2007-2017 Inc 5000 company list
Inc_10year_csv ="Resources/inc5000_all10years.csv"


### Read the files

In [3]:
# Read csv file
# Read 2019 Inc 5000 company list
Inc_2019_df = pd.read_csv(Inc_2019_csv)

# Read 2018 Inc 5≠000 company list
Inc_2018_df = pd.read_csv(Inc_2018_csv)


# Read 2007-2017 Inc 5000 company list
Inc_10year_df = pd.read_csv(Inc_10year_csv,encoding='cp1252')

## Scraping Financial Times Americas Fastest Growing Companies 2020

In [4]:
ft_url = 'https://www.ft.com/americas-fastest-growing-companies-2020'

In [5]:
ft_tables= pd.read_html(ft_url)

In [6]:
type(ft_tables)

list

In [7]:
len(ft_tables)

1

In [8]:
ft_tables_df = ft_tables[0]
ft_tables_df.head()

,Rank,Name,Country,FT category,Absolute growth rate [in %],Compound annual growth rate (CAGR) [in %],Revenue 2018 [in $m],Revenue 2015 [in $m],Number of employees 2018,Founding Year
0,1,Niantic*,US,Games industry,180306.6,1117.4,790.18,0.44,426,2015
1,2,UiPath,US,Technology,37463.5,621.5,155.00,0.41,3200,2005
2,3,Publisher First (Freestar)*,US,Advertising,36680.1,616.5,36.86,0.10,43,2015
3,4,FreightWise*,US,Transport,30547.9,574.2,33.64,0.11,39,2015
4,5,Veggie Noodle Co.*,US,Food & Beverage,24074.8,523.0,24.90,0.10,175,2015


## API Call for Scraping Growjo top 1000 fastest growing companies in 2020.

In [9]:
base_url = "https://growjo.com/api?url="

ranking = []
estimated_revenues = []
job_openings = []
company_name = []
city = []
country = []
state = []
employees = []
founded = []
linkedin_url = []
url = []
industry = []
total_funding = []

for domain in company_domain:
    target_url = base_url + domain
    data = requests.get(target_url)
    try:
        response = data.json()
    except ValueError:
        pass
    ranking.append(response['ranking'])
    estimated_revenues.append(response['estimated_revenues'])
    job_openings.append(response['job_openings'])
    company_name.append(response['company_name'])
    city.append(response['city'])
    country.append(response['country'])
    state.append(response['state'])
    employees.append(response['employees'])
    founded.append(response['founded'])
    url.append(response['url'])
    industry.append(response['industry'])
    total_funding.append(response['total_funding'])
    

In [10]:
growjo_df = pd.DataFrame({
    "ranking":ranking,
    "estimated_revenues":estimated_revenues,
    "job_openings":job_openings,
    "company_name": company_name,
    "city":city,
    "country":country,
    "state":state,
    "employees":employees,
    "founded":founded,
    "url":url,
    "industry":industry,
    "total_funding":total_funding
})

# Remove duplicate values
clean_growjo_df=growjo_df.drop_duplicates()
clean_growjo_df.head()

,ranking,estimated_revenues,job_openings,company_name,city,country,state,employees,founded,url,industry,total_funding
0,1.0,36450000.0,6,LetsGetChecked,New York,United States,NY,135,2014.0,letsgetchecked.com,Hospital/Healthcare,$113M
1,701.0,13120000.0,1,100 Thieves,Los Angeles,United States,CA,64,2017.0,100thieves.com,Entertainment,None
2,832.0,97340000.0,66,10X Genomics,Pleasanton,United States,CA,628,2012.0,10xgenomics.com,Biotech,$242.6M
3,124.0,29870000.0,1,15Five,San Francisco,United States,CA,206,2011.0,15five.com,Tech Services,$42.1M
4,941.0,10793000.0,1,2ULaundry,Charlotte,United States,NC,43,2015.0,2ulaundry.com,Consumer,None


# Transforming

### Cleaning the data 

### Cleaning 2019 Inc 5000 company list

In [14]:
# Drop column we don't need
drop_column_2019=Inc_2019_df.drop(['Profile','_ - previous_workers','_ - founded','_ - metro','url','_ - yrs_on_list'], axis=1)

# Add the year column 
drop_column_2019['Year']= '2019'

# Rename columns 
rename_df_2019= drop_column_2019.rename(columns={"_ - rank":"Rank","name":"Company_Name","state":"State","_ - revenue":"Revenue $","_ - growth":"Growth","_ - industry":"Industry","_ - workers":"Number_of_Employees","city":"City"})


# re-Order the data to suitable format

cleaned_2019_df =rename_df_2019[['Rank','Year','Company_Name','Industry','Number_of_Employees','Revenue $','Growth','City','State']]
cleaned_2019_df.head()


,Rank,Year,Company_Name,Industry,Number_of_Employees,Revenue $,Growth,City,State
0,1,2019,Freestar,Advertising & Marketing,40.0,36.9 Million,36680.3882,Phoenix,AZ
1,2,2019,FreightWise,Logistics & Transportation,39.0,33.6 Million,30547.9317,Brentwood,TN
2,3,2019,Cece's Veggie Co.,Food & Beverage,190.0,24.9 Million,23880.4852,Austin,TX
3,4,2019,LadyBoss,Consumer Products & Services,57.0,32.4 Million,21849.8925,Albuquerque,NM
4,5,2019,Perpay,Retail,25.0,22.5 Million,18166.4070,Philadelphia,PA


### Cleaning 2018 Inc 5000 company list

In [15]:
# Drop column we don't need

drop_column_2018=Inc_2018_df.drop(['_ - id','_ - zipcode','_ - ifmid','_ - latitude','_ - longitude','_ - website','_ - state_l','_ - metrocode','_ - ifiid','_ - previous_workers','_ - metro','_ - founded','_ - url','_ - partner_lists - partner_lists','_ - yrs_on_list'], axis=1)

# Add the year column 
drop_column_2018['Year']= '2018'

# Rename columns 
rename_df_2018=drop_column_2018.rename(columns={"_ - rank":"Rank","_ - company":"Company_Name","_ - state_s":"State","_ - revenue":"Revenue $","_ - growth":"Growth","_ - industry":"Industry","_ - workers":"Number_of_Employees","_ - state_l":"State","_ - city":"City"})
rename_df_2018.head()

# Normalizing revenue in to mellions 
rename_df_2018['Revenue $']= (rename_df_2018['Revenue $']/1000000).apply(lambda x: '{:,.1f} Million'.format(x))


cleaned_2018_df =rename_df_2018[['Rank','Year','Company_Name','Industry','Number_of_Employees','Revenue $','Growth','City','State']]
cleaned_2018_df.head()

,Rank,Year,Company_Name,Industry,Number_of_Employees,Revenue $,Growth,City,State
0,1.0,2018,SwanLeap,Logistics & Transportation,49.0,99.0 Million,75660.8425,Madison,WI
1,2.0,2018,PopSockets,Consumer Products & Services,118.0,168.8 Million,71423.7620,Boulder,CO
2,3.0,2018,Home Chef,Food & Beverage,865.0,255.0 Million,60165.5058,Chicago,IL
3,4.0,2018,Velocity Global,Business Products & Services,55.0,49.2 Million,39816.5093,Denver,CO
4,5.0,2018,DEPCOM Power,Energy,104.0,219.6 Million,38962.9022,Scottsdale,AZ


### Cleaning 10 year (2007-2017) Inc 5000 company list

In [20]:
# Rename columns 

drop_column_10year_df=Inc_10year_df.drop(['_ - state_l','_ - metro','_ - yrs_on_list'],axis=1)

# Add the year column 
rename_10year_df=drop_column_10year_df.rename(columns={"year":"Year","_ - rank":"Rank","_ - company":"Company_Name","_ - website":"Company_Website","_ - state_s":"State","_ - revenue":"Revenue $","_ - growth":"Growth","_ - industry":"Industry","_ - workers":"Number_of_Employees","_ - founded":"Founded_Year","_ - city":"City"})


# Normalizing revenue in to mellions 
rename_10year_df['Revenue $']= (rename_10year_df['Revenue $']/1000000).apply(lambda x: '{:,.1f} Million'.format(x))

cleaned_10year_df =rename_10year_df[['Rank','Year','Company_Name','Industry','Number_of_Employees','Revenue $','Growth','City','State']]
cleaned_10year_df.head()


,Rank,Year,Company_Name,Industry,Number_of_Employees,Revenue $,Growth,City,State
0,1,2016,Loot Crate,Consumer Products & Services,218.0,116.2 Million,66788.5962,Los Angeles,CA
1,2,2016,Paint Nite,Consumer Products & Services,100.0,55.0 Million,36555.2472,Somerville,MA
2,3,2016,CalCom Solar,Energy,47.0,33.5 Million,31633.5448,Visalia,CA
3,4,2016,eLuxurySupply.com,Retail,82.0,30.7 Million,23619.7198,Evansville,IN
4,5,2016,Company.com,Business Products & Services,48.0,33.4 Million,23486.8894,Atlanta,GA


#### Combine all data in to a single dataframe

In [92]:
# combine all data in to a single dataframe

combine_data =[cleaned_2018_df,cleaned_2019_df,cleaned_10year_df]

Inc_5000_df= pd.concat(combine_data)
Inc_5000_df.head()

,Rank,Year,Company_Name,Industry,Number_of_Employees,Revenue $,Growth,City,State
0,1.0,2018,SwanLeap,Logistics & Transportation,49.0,99.0 Million,75660.8425,Madison,WI
1,2.0,2018,PopSockets,Consumer Products & Services,118.0,168.8 Million,71423.7620,Boulder,CO
2,3.0,2018,Home Chef,Food & Beverage,865.0,255.0 Million,60165.5058,Chicago,IL
3,4.0,2018,Velocity Global,Business Products & Services,55.0,49.2 Million,39816.5093,Denver,CO
4,5.0,2018,DEPCOM Power,Energy,104.0,219.6 Million,38962.9022,Scottsdale,AZ


In [93]:
# Add unique id for company name 
id_df=Inc_5000_df.groupby(['Company_Name'], sort=True).ngroup().apply('{:006}'.format)

Inc_5000_df['Company_id']=id_df

Inc_5000_df.head()

,Rank,Year,Company_Name,Industry,Number_of_Employees,Revenue $,Growth,City,State,Company_id
0,1.0,2018,SwanLeap,Logistics & Transportation,49.0,99.0 Million,75660.8425,Madison,WI,019877
1,2.0,2018,PopSockets,Consumer Products & Services,118.0,168.8 Million,71423.7620,Boulder,CO,015956
2,3.0,2018,Home Chef,Food & Beverage,865.0,255.0 Million,60165.5058,Chicago,IL,009633
3,4.0,2018,Velocity Global,Business Products & Services,55.0,49.2 Million,39816.5093,Denver,CO,022286
4,5.0,2018,DEPCOM Power,Energy,104.0,219.6 Million,38962.9022,Scottsdale,AZ,005706


## Analysis